In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, io
from pathlib import Path
from PIL import Image
import os


In [2]:
# Define transformations for the input images
data_transform = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor(),
])


In [5]:
# Create a custom dataset class
class ConcreteCrackDataset(Dataset):
    def __init__(self, positive_dir, negative_dir, transform=None):
        self.positive_dir = positive_dir
        self.negative_dir = negative_dir
        self.transform = transform
        self.classes = ['negative', 'positive']
        self.image_paths = self.load_image_paths()

    def load_image_paths(self):
        image_paths = []
        for class_name in self.classes:
            class_path = getattr(self, f'{class_name}_dir')
            class_images = [image for image in class_path.glob('*.jpg')]
            image_paths.extend(class_images)
        return image_paths

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = 1 if 'positive' in img_path.name else 0  # 1 for positive, 0 for negative

        return image, label

In [6]:
# Define dataset paths
positive_dir = Path('C:\\Users\\Madhava Reddy\\Downloads\\concrete crack image classification\\Concrete Crack Images for Classification\\Positive')
negative_dir = Path('C:\\Users\\Madhava Reddy\\Downloads\\concrete crack image classification\\Concrete Crack Images for Classification\\Negative')


In [7]:
# Create dataset and DataLoader
concrete_crack_dataset = ConcreteCrackDataset(positive_dir, negative_dir, transform=data_transform)
dataloader = DataLoader(concrete_crack_dataset, batch_size=32, shuffle=True)

In [8]:
# Display the first batch of images and labels
for images, labels in dataloader:
    print(images.shape)  # Print the shape of the images tensor
    print(labels)  # Print the labels tensor
    break  # Break after the first batch for brevity

torch.Size([32, 3, 227, 227])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from pathlib import Path

In [10]:
# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 2)  # Two classes: negative and positive

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.flatten(x)
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x

In [11]:
# Instantiate the model, loss function, and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [13]:
# Train the model
num_epochs = 1
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(dataloader)}')

KeyboardInterrupt: 

In [ ]:
# Save the trained model
torch.save(model.state_dict(), 'path/to/trained_model.pth')

In [ ]:
# Prediction
model.eval()
new_image_path = 'path/to/new_image.jpg'  # Replace with the path to your new image
new_image = Image.open(new_image_path).convert('RGB')
new_image = data_transform(new_image).unsqueeze(0)

In [ ]:
with torch.no_grad():
    output = model(new_image)
    predicted_class = torch.argmax(output, dim=1).item()
if predicted_class == 0:
    print('Predicted class: Negative (No crack)')
else:
    print('Predicted class: Positive (Crack)')